In [47]:
import pandas as pd
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
import sqlalchemy as sa
from sqlalchemy import create_engine
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [48]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Gaoretelelwe'

connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

sql_query = "select * from [earmark-stoki-stoki]"

In [49]:
df_bank_statements = pd.read_sql_query(sql_query, connection)
df_bank_statements.Category = df_bank_statements.Category.str.upper()
df_bank_statements.head(25)

,Transaction_Date,Posting_Date,Description,Debits,Credits,Category
0,2020-01-27,2020-01-27,BUDGET VAP 610117103 JAN PREM,411.43,None,OTHER
1,2020-01-25,2020-01-26,AMZN DIGITAL*L59CI12Z3 888-802-3080 US,36.19,None,ENTERTAINMENT & EAT OUT
2,2020-01-24,2020-01-25,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,40,None,ENTERTAINMENT & EAT OUT
3,2020-01-23,2020-01-25,INTELLEPARK ROSEBANK ZA,8,None,TRANSPORT
4,2020-01-25,2020-01-25,MTN DATA BUNDLES,499,None,COMMUNICATION
5,2020-01-25,2020-01-25,TSHIFHIWA RENT,None,4950,SAVING AND INVESTMENT
6,2020-01-23,2020-01-24,MINTY'S SASOL ROSEBANK PARKWOOD ZA,656.42,None,TRANSPORT
7,2020-01-23,2020-01-24,SEATTLE RENNIES HOUSE JOHANNESBURG ZA,31.5,None,ENTERTAINMENT & EAT OUT
8,2020-01-22,2020-01-23,WOOLWORTHS PARKTOWN QUA JOHANNESBURG ZA,192.59,None,FOOD
9,2020-01-25,2020-01-26,AFTER HOURS VET CLINIC PARKTOWN NORT ZA,535,None,HOME


In [50]:
# Lables
y = df_bank_statements['Category']

# Features
X = df_bank_statements.drop(['Category'], axis=1)

X

,Transaction_Date,Posting_Date,Description,Debits,Credits
0,2020-01-27,2020-01-27,BUDGET VAP 610117103 JAN PREM,411.43,None
1,2020-01-25,2020-01-26,AMZN DIGITAL*L59CI12Z3 888-802-3080 US,36.19,None
2,2020-01-24,2020-01-25,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,40,None
3,2020-01-23,2020-01-25,INTELLEPARK ROSEBANK ZA,8,None
4,2020-01-25,2020-01-25,MTN DATA BUNDLES,499,None
...,...,...,...,...,...
938,NaT,2020-02-02,IBANK PAYMENT,200,None
939,NaT,2020-02-02,POS PURCHASE BP Filling Sta,81.6,None
940,NaT,2020-02-02,POS PURCHASE BP Filling Sta,28,None
941,NaT,2020-02-01,ACB DEBIT:EXTERNAL VIRGIN ACT,297.5,None


In [51]:
y_unique = y.value_counts()
y_unique = pd.DataFrame(y_unique)
y_unique = y_unique.reset_index(drop=False)
y_unique.drop(['Category'], axis=1, inplace=True) 
y_unique['y_id'] = y_unique.index

y_unique

,index,y_id
0,TRANSPORT,0
1,FEES,1
2,ENTERTAINMENT & EAT OUT,2
3,COMMUNICATION,3
4,FOOD,4
5,HOME,5
6,OTHER,6
7,SHORT-TERM CREDIT,7
8,LIFE AND HEALTH,8
9,SAVING AND INVESTMENT,9


In [52]:
y =  pd.DataFrame(y)
y.head(25)

,Category
0,OTHER
1,ENTERTAINMENT & EAT OUT
2,ENTERTAINMENT & EAT OUT
3,TRANSPORT
4,COMMUNICATION
5,SAVING AND INVESTMENT
6,TRANSPORT
7,ENTERTAINMENT & EAT OUT
8,FOOD
9,HOME


In [53]:
mapping = dict(y_unique[['index', 'y_id']].values)
y['Item_ID'] = y.Category.map(mapping)
y.head(25)

,Category,Item_ID
0,OTHER,6
1,ENTERTAINMENT & EAT OUT,2
2,ENTERTAINMENT & EAT OUT,2
3,TRANSPORT,0
4,COMMUNICATION,3
5,SAVING AND INVESTMENT,9
6,TRANSPORT,0
7,ENTERTAINMENT & EAT OUT,2
8,FOOD,4
9,HOME,5


In [54]:
y.drop(['Category'], axis=1, inplace=True)
y.head(25)

,Item_ID
0,6
1,2
2,2
3,0
4,3
5,9
6,0
7,2
8,4
9,5


In [55]:
y.tail()

,Item_ID
938,13
939,0
940,0
941,8
942,1


In [56]:
X.head(25)

,Transaction_Date,Posting_Date,Description,Debits,Credits
0,2020-01-27,2020-01-27,BUDGET VAP 610117103 JAN PREM,411.43,None
1,2020-01-25,2020-01-26,AMZN DIGITAL*L59CI12Z3 888-802-3080 US,36.19,None
2,2020-01-24,2020-01-25,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,40,None
3,2020-01-23,2020-01-25,INTELLEPARK ROSEBANK ZA,8,None
4,2020-01-25,2020-01-25,MTN DATA BUNDLES,499,None
5,2020-01-25,2020-01-25,TSHIFHIWA RENT,None,4950
6,2020-01-23,2020-01-24,MINTY'S SASOL ROSEBANK PARKWOOD ZA,656.42,None
7,2020-01-23,2020-01-24,SEATTLE RENNIES HOUSE JOHANNESBURG ZA,31.5,None
8,2020-01-22,2020-01-23,WOOLWORTHS PARKTOWN QUA JOHANNESBURG ZA,192.59,None
9,2020-01-25,2020-01-26,AFTER HOURS VET CLINIC PARKTOWN NORT ZA,535,None


In [57]:
nltk.download('stopwords')
nltk.download('wordnet')

# create stop words and lemmatizer instances
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

# create function to clean text
def clean_text(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    text = [lemmatizer.lemmatize(word) for word in text.split(' ')]
    text = [word for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

new_features = df_bank_statements['Description'].apply(lambda x: clean_text(x))

new_features

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaoretelelwe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gaoretelelwe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0                     budget vap 610117103 jan prem
1                amzn digitall59ci12z3 8888023080 u
2            flm rennie house eater braamfontein za
3                           intellepark rosebank za
4                                   mtn data bundle
                           ...                     
938                                  ibank payment 
939        po purchase               bp filling sta
940    po purchase                   bp filling sta
941          acb debitexternal           virgin act
942                                  management fee
Name: Description, Length: 943, dtype: object

In [58]:
X

,Transaction_Date,Posting_Date,Description,Debits,Credits
0,2020-01-27,2020-01-27,BUDGET VAP 610117103 JAN PREM,411.43,None
1,2020-01-25,2020-01-26,AMZN DIGITAL*L59CI12Z3 888-802-3080 US,36.19,None
2,2020-01-24,2020-01-25,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,40,None
3,2020-01-23,2020-01-25,INTELLEPARK ROSEBANK ZA,8,None
4,2020-01-25,2020-01-25,MTN DATA BUNDLES,499,None
...,...,...,...,...,...
938,NaT,2020-02-02,IBANK PAYMENT,200,None
939,NaT,2020-02-02,POS PURCHASE BP Filling Sta,81.6,None
940,NaT,2020-02-02,POS PURCHASE BP Filling Sta,28,None
941,NaT,2020-02-01,ACB DEBIT:EXTERNAL VIRGIN ACT,297.5,None


In [59]:
df_bank_statements['Description'] = df_bank_statements['Description'].str.lower()

In [60]:
y_unique

,index,y_id
0,TRANSPORT,0
1,FEES,1
2,ENTERTAINMENT & EAT OUT,2
3,COMMUNICATION,3
4,FOOD,4
5,HOME,5
6,OTHER,6
7,SHORT-TERM CREDIT,7
8,LIFE AND HEALTH,8
9,SAVING AND INVESTMENT,9


In [61]:
y_unique_dict = y_unique.to_dict()['index']
y_unique_dict

{0: 'TRANSPORT',
 1: 'FEES',
 2: 'ENTERTAINMENT & EAT OUT',
 3: 'COMMUNICATION',
 4: 'FOOD',
 5: 'HOME',
 6: 'OTHER',
 7: 'SHORT-TERM CREDIT',
 8: 'LIFE AND HEALTH',
 9: 'SAVING AND INVESTMENT',
 10: 'GIFTS & DONATIONS',
 11: 'EDUCATION',
 12: 'INCOME',
 13: 'GIFTS AND DONATIONS',
 14: 'INTEREST'}

In [62]:
import math

category_dict = {}

for key in y_unique_dict: 
    code = int(key)
    category = y_unique_dict[key]
    category_dict[category] = code # math.trunc(code)

df_bank_statements.iloc[891]

Transaction_Date                                  2020-02-02 00:00:00
Posting_Date                                      2020-02-02 00:00:00
Description         pos purchase- woolworths campus squarjohannesb...
Debits                                                           -800
Credits                                                          None
Category                                                         HOME
Name: 891, dtype: object

In [63]:
df_bank_statements['b_Category']  = df_bank_statements['Category'].map(category_dict)
#df_bank_statements.b_Category = df_bank_statements.b_Category.astype(str)
#df_bank_statements["b_Category"] = df_bank_statements["b_Category"].astype(int, errors='ignore')
df_bank_statements.head(25)

,Transaction_Date,Posting_Date,Description,Debits,Credits,Category,b_Category
0,2020-01-27,2020-01-27,budget vap 610117103 jan prem,411.43,None,OTHER,6
1,2020-01-25,2020-01-26,amzn digital*l59ci12z3 888-802-3080 us,36.19,None,ENTERTAINMENT & EAT OUT,2
2,2020-01-24,2020-01-25,flm rennie house eater braamfontein za,40,None,ENTERTAINMENT & EAT OUT,2
3,2020-01-23,2020-01-25,intellepark rosebank za,8,None,TRANSPORT,0
4,2020-01-25,2020-01-25,mtn data bundles,499,None,COMMUNICATION,3
5,2020-01-25,2020-01-25,tshifhiwa rent,None,4950,SAVING AND INVESTMENT,9
6,2020-01-23,2020-01-24,minty's sasol rosebank parkwood za,656.42,None,TRANSPORT,0
7,2020-01-23,2020-01-24,seattle rennies house johannesburg za,31.5,None,ENTERTAINMENT & EAT OUT,2
8,2020-01-22,2020-01-23,woolworths parktown qua johannesburg za,192.59,None,FOOD,4
9,2020-01-25,2020-01-26,after hours vet clinic parktown nort za,535,None,HOME,5


In [64]:
Y = df_bank_statements['b_Category']
X = new_features

print(Y)
print(X)

0       6
1       2
2       2
3       0
4       3
       ..
938    13
939     0
940     0
941     8
942     1
Name: b_Category, Length: 943, dtype: int64
0                     budget vap 610117103 jan prem
1                amzn digitall59ci12z3 8888023080 u
2            flm rennie house eater braamfontein za
3                           intellepark rosebank za
4                                   mtn data bundle
                           ...                     
938                                  ibank payment 
939        po purchase               bp filling sta
940    po purchase                   bp filling sta
941          acb debitexternal           virgin act
942                                  management fee
Name: Description, Length: 943, dtype: object


In [65]:
X

0                     budget vap 610117103 jan prem
1                amzn digitall59ci12z3 8888023080 u
2            flm rennie house eater braamfontein za
3                           intellepark rosebank za
4                                   mtn data bundle
                           ...                     
938                                  ibank payment 
939        po purchase               bp filling sta
940    po purchase                   bp filling sta
941          acb debitexternal           virgin act
942                                  management fee
Name: Description, Length: 943, dtype: object

In [66]:
count_vectorizer = CountVectorizer(decode_error = 'ignore')
X_vectorized = count_vectorizer.fit_transform(X)

X_vectorized

<943x724 sparse matrix of type '<class 'numpy.int64'>'
	with 3937 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y, test_size=0.3, random_state=50)

In [68]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(random_state=50)

In [69]:
y_train.head(25)

216    0
530    2
417    0
652    0
635    0
180    4
554    3
246    1
577    1
858    1
638    3
493    0
108    5
541    0
806    1
87     6
825    3
168    0
455    1
110    6
608    0
94     9
892    5
782    5
851    1
Name: b_Category, dtype: int64

In [70]:
knn.fit(X_train, y_train)
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=50, splitter='best')

In [71]:
pred_knn = knn.predict(X_test)
pred_tree = tree.predict(X_test)

In [72]:
pred_knn

array([ 1,  0,  9,  3,  1,  9,  4,  0,  9,  0,  0,  3,  9,  1,  2,  0,  2,
        9,  0,  0,  0,  9,  2,  2,  9,  2,  2,  0,  0,  0,  2,  9,  1,  9,
        4,  2,  0,  0,  9,  0,  0,  3,  1,  0,  9,  1,  1,  0,  9,  0,  1,
        9,  2,  9,  1,  9,  2,  2,  4,  9,  1,  0,  0,  0,  9,  0,  2,  4,
        5,  1,  1,  0,  1,  0,  0,  0,  9,  0,  0,  2,  9,  2,  1,  1,  2,
        0,  1,  9,  1,  0,  0,  0,  0,  2,  9,  9,  1,  1,  0,  2,  9,  1,
        0,  9,  2,  9,  0,  4,  2,  1,  2,  0,  0,  0,  0,  9,  9,  0,  9,
        9,  1,  1,  0,  9,  9,  9,  0,  0,  2,  1,  9,  0,  1,  0,  0,  0,
        9,  0,  1,  3,  1,  0,  0,  3,  0,  9,  2,  9,  3,  3,  9,  9,  0,
        1,  9,  9,  9,  0,  0,  1,  9,  1,  2,  2,  9,  1,  0,  2,  5,  1,
        5,  9,  1,  0,  2,  9,  9,  0,  0,  2,  3,  2, 10,  0,  0,  2,  0,
        9,  0,  2,  0,  1,  9,  1,  0,  0,  0,  0,  4,  0,  2,  1,  0,  0,
        0, 10,  3,  9,  2,  0,  9,  3,  1,  1,  3,  0,  3,  1,  4,  9,  0,
        0,  0,  1,  5,  0

In [73]:
pred_tree

array([ 1,  0,  9,  3,  1,  9,  2,  0,  7,  2,  0,  3,  9,  1,  2,  0,  2,
        9,  0,  0,  0,  0,  2,  2, 10,  2,  2,  2,  0,  0,  2,  9,  9,  0,
        4,  2,  0,  0,  1,  0,  0,  3,  1,  0,  5,  1,  1,  0,  9,  0,  9,
        9,  4,  9,  1,  9,  0,  8,  2,  2,  1,  0,  0,  0,  1,  0,  2,  2,
        5,  1,  1,  0,  1,  0,  0,  0,  2,  0,  2,  2,  5,  2,  1,  1,  2,
        0,  1,  9,  1,  0,  2,  0,  0,  2,  9,  3,  1,  1,  0,  2,  9,  1,
        0, 10,  2,  9,  0,  2,  2,  1,  2,  0,  0,  0,  0,  7,  9,  0,  9,
        9,  1,  1,  0,  2,  7,  9,  0,  0,  2,  1,  2,  0,  6,  0,  0,  0,
        2,  0,  1,  3,  1,  0,  2,  3,  2,  2,  2, 10,  3,  3,  9,  6,  0,
        1,  9, 10,  0,  0,  0,  1,  1,  1,  2,  2,  7,  1,  0,  5,  5,  6,
        5,  2,  1,  0,  6,  2,  1,  0,  2,  2,  3,  2, 10,  0,  0,  2,  0,
        2,  0,  2,  0,  6,  2,  1,  0,  0,  0,  2,  2,  0,  2,  1,  0,  0,
        0, 10,  3,  9,  2,  0,  6,  3,  1,  1,  3,  0,  3,  1,  2,  2,  0,
        0,  0,  1,  5,  0

In [74]:
y_unique

,index,y_id
0,TRANSPORT,0
1,FEES,1
2,ENTERTAINMENT & EAT OUT,2
3,COMMUNICATION,3
4,FOOD,4
5,HOME,5
6,OTHER,6
7,SHORT-TERM CREDIT,7
8,LIFE AND HEALTH,8
9,SAVING AND INVESTMENT,9


In [75]:
df_predict_knn = pd.DataFrame({'prediction': pred_knn})
df_predict_tree = pd.DataFrame({'prediction': pred_tree})

In [39]:
df_predict_knn

,prediction
0,1
1,0
2,6
3,3
4,1
...,...
278,4
279,3
280,2
281,0


In [40]:
df_predict_tree

,prediction
0,1
1,0
2,6
3,3
4,1
...,...
278,4
279,3
280,2
281,0


In [32]:
X_test

<283x22 sparse matrix of type '<class 'numpy.int64'>'
	with 437 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_knn))
print(classification_report(y_test, pred_tree))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       101
           1       1.00      1.00      1.00        55
           2       1.00      1.00      1.00        54
           3       1.00      1.00      1.00        18
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00         3

    accuracy                           1.00       283
   macro avg       1.00      1.00      1.00       283
weighted avg       1.00      1.00      1.00       283

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       101
           1       1.00 

In [34]:
y_unique

,index,y_id
0,TRANSPORT,0
1,FEES,1
2,ENTERTAINMENT & EAT OUT,2
3,COMMUNICATION,3
4,FOOD,4
5,HOME,5
6,OTHER,6
7,SHORT-TERM CREDIT,7
8,LIFE AND HEALTH,8
9,SAVING AND INVESTMENT,9


In [41]:
mapping = dict(y_unique[['y_id','index']].values)
df_predict_knn['ItemCategory_New'] = df_predict_knn.prediction.map(mapping)
df_predict_knn

,prediction,ItemCategory_New
0,1,FEES
1,0,TRANSPORT
2,6,OTHER
3,3,COMMUNICATION
4,1,FEES
...,...,...
278,4,FOOD
279,3,COMMUNICATION
280,2,ENTERTAINMENT & EAT OUT
281,0,TRANSPORT


In [42]:
mapping = dict(y_unique[['y_id','index']].values)
df_predict_tree['ItemCategory_New'] = df_predict_tree.prediction.map(mapping)
df_predict_tree

,prediction,ItemCategory_New
0,1,FEES
1,0,TRANSPORT
2,6,OTHER
3,3,COMMUNICATION
4,1,FEES
...,...,...
278,4,FOOD
279,3,COMMUNICATION
280,2,ENTERTAINMENT & EAT OUT
281,0,TRANSPORT


In [45]:
type(X_test)

scipy.sparse.csr.csr_matrix